#### oneshot

In [1]:
from functions import *
model, tokenizer = FastModel.from_pretrained(model_name = "unsloth/gemma-3-4b-pt-unsloth-bnb-4bit",
                                        max_seq_length = 8192,
                                        load_in_4bit = True,
                                    )

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-19 13:53:14 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.3.19: Fast Gemma3 patching. Transformers: 4.50.0.dev0. vLLM: 0.8.2.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.642 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [2]:
custom_vocab_size = 48

In [3]:
pretrained_tokenizer = tokenizer.tokenizer
pretrained_model = model

In [28]:
pretrained_embedding = pretrained_model.get_input_embeddings()
pretrained_bos_id = pretrained_tokenizer.bos_token_id
pretrained_eos_id = pretrained_tokenizer.eos_token_id

# Initialize custom embedding with the same embedding dimension as pre-trained
embedding_dim = pretrained_embedding.embedding_dim
custom_embedding = nn.Embedding(custom_vocab_size, embedding_dim)
with torch.no_grad():
    custom_embedding.weight.normal_(mean=0.0, std=0.0192)
# Set weights for special tokens
custom_embedding.weight.data[10] = pretrained_embedding.weight.data[pretrained_bos_id]  # BOS_X
custom_embedding.weight.data[12] = pretrained_embedding.weight.data[pretrained_bos_id]  # BOS_Y
custom_embedding.weight.data[11] = pretrained_embedding.weight.data[pretrained_eos_id]  # EOS_X
custom_embedding.weight.data[13] = pretrained_embedding.weight.data[pretrained_eos_id]  # EOS_Y

In [29]:
for i in range(1, 30):
    # Encode the integer directly
    txt = str(i % 10) # Assuming single token output for each integer. only true for 1~9, hence %.
    token_id = pretrained_tokenizer.encode(txt, add_special_tokens=False)
    assert len(token_id) == 1, f"Tokenization failed for {txt}: {token_id}"
    custom_embedding.weight.data[16 + i] = pretrained_embedding.weight.data[token_id[0]]
    # 

In [30]:
custom_embedding.weight.std(1)

tensor([0.0192, 0.0188, 0.0196, 0.0196, 0.0190, 0.0189, 0.0188, 0.0189, 0.0193,
        0.0191, 0.0192, 0.0192, 0.0192, 0.0192, 0.0191, 0.0191, 0.0191, 0.0192,
        0.0192, 0.0192, 0.0192, 0.0192, 0.0192, 0.0192, 0.0192, 0.0192, 0.0192,
        0.0192, 0.0192, 0.0192, 0.0192, 0.0192, 0.0192, 0.0192, 0.0192, 0.0192,
        0.0192, 0.0192, 0.0192, 0.0192, 0.0192, 0.0192, 0.0192, 0.0192, 0.0192,
        0.0192, 0.0190, 0.0189], grad_fn=<StdBackward0>)

In [31]:
torch.save(custom_embedding.state_dict(), '../../Model/gemma48.pth')

#### Causal

In [ ]:
from functions import *
model, tokenizer = FastModel.from_pretrained(
                                        model_name = "unsloth/gemma-3-4b-pt-unsloth-bnb-4bit",
                                        # model_name = "unsloth/Qwen3-8B-unsloth-bnb-4bit",
                                        max_seq_length = 8192,
                                        load_in_4bit = True,
                                    )

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.7: Fast Qwen3 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.642 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
custom_vocab_size = 16

In [ ]:
pretrained_tokenizer = tokenizer.tokenizer
# pretrained_tokenizer = tokenizer
pretrained_model = model

In [ ]:
pretrained_embedding = pretrained_model.get_input_embeddings()
pretrained_bos_id = pretrained_tokenizer.bos_token_id
pretrained_eos_id = pretrained_tokenizer.eos_token_id

# Initialize custom embedding with the same embedding dimension as pre-trained
embedding_dim = pretrained_embedding.embedding_dim
custom_embedding = nn.Embedding(custom_vocab_size, embedding_dim)
with torch.no_grad():
    custom_embedding.weight.normal_(mean=0.0, std=0.0192)
# Set weights for special tokens
custom_embedding.weight.data[10] = pretrained_embedding.weight.data[pretrained_bos_id]  # BOS_X
custom_embedding.weight.data[11] = pretrained_embedding.weight.data[pretrained_eos_id]  # EOS_X
custom_embedding.weight.data[13] = pretrained_embedding.weight.data[pretrained_bos_id]  # BOS_Y
custom_embedding.weight.data[14] = pretrained_embedding.weight.data[pretrained_eos_id]  # EOS_Y
custom_embedding.weight.data[12] = pretrained_tokenizer.encode("\n", add_special_tokens=False)[0]  # Line break

In [6]:
custom_embedding.weight.std(1)

tensor([0.0189, 0.0192, 0.0197, 0.0195, 0.0190, 0.0193, 0.0197, 0.0193, 0.0191,
        0.0191, 0.0192, 0.0192, 0.0000, 0.0192, 0.0192, 0.0192],
       grad_fn=<StdBackward0>)

In [7]:
torch.save(custom_embedding.state_dict(), '../../Model/gemma16.pth')

#### VLM

In [1]:
from functions import *
model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-pt-unsloth-bnb-4bit",
    max_seq_length = 8192, # Choose any for long context!
    load_in_4bit = True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-24 08:44:26 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.3.19: Fast Gemma3 patching. Transformers: 4.50.0.dev0. vLLM: 0.8.2.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.642 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [2]:
model.language_model.lm_head.load_state_dict(torch.load("/home/zhenlan/Desktop/Projects/ARC2/Model/lm_heads_weights_VLM.pth"))

<All keys matched successfully>

In [3]:
custom_vocab_size = 24
pretrained_tokenizer = tokenizer.tokenizer
pretrained_model = model
pretrained_embedding = pretrained_model.get_input_embeddings()
embedding_dim = pretrained_embedding.embedding_dim
custom_embedding = nn.Embedding(custom_vocab_size, embedding_dim)
colors = [" red", " blue", " green", " yellow", " orange", " purple", " white", " cyan", " gray", " brown"]
for i, color in enumerate(colors):
    # Encode the color directly
    token_id = pretrained_tokenizer.encode(color, add_special_tokens=False)
    assert len(token_id) == 1, f"Tokenization failed for {color}: {token_id}"
    print(f"Token ID for '{color}': {token_id[0]}")
    custom_embedding.weight.data[i] = pretrained_embedding.weight.data[token_id[0]]
token_id = pretrained_tokenizer.encode('<bos>', add_special_tokens=False)
custom_embedding.weight.data[10] = pretrained_embedding.weight.data[token_id[0]]
print(f"Token ID for '<bos>': {token_id[0]}, new id: {10}")
token_id = pretrained_tokenizer.encode(' input', add_special_tokens=False)
custom_embedding.weight.data[11] = pretrained_embedding.weight.data[token_id[0]]
print(f"Token ID for ' input': {token_id[0]}, new id: {11}")
token_id = pretrained_tokenizer.encode(' output', add_special_tokens=False)
custom_embedding.weight.data[12] = pretrained_embedding.weight.data[token_id[0]]
print(f"Token ID for ' output': {token_id[0]}, new id: {12}")
token_id = pretrained_tokenizer.encode('\n', add_special_tokens=False)
custom_embedding.weight.data[13] = pretrained_embedding.weight.data[token_id[0]]
print(f"Token ID for '\\n': {token_id[0]}, new id: {13}")
token_id = pretrained_tokenizer.encode('\n\n\n', add_special_tokens=False)
custom_embedding.weight.data[14] = pretrained_embedding.weight.data[token_id[0]]
print(f"Token ID for '\\n\\n\\n': {token_id[0]}, new id: {14}")
token_id = pretrained_tokenizer.encode('<start_of_image>', add_special_tokens=False)
custom_embedding.weight.data[15] = pretrained_embedding.weight.data[token_id[0]]
print(f"Token ID for '<start_of_image>': {token_id[0]}, new id: {15}")
token_id = pretrained_tokenizer.encode('<image_soft_token>', add_special_tokens=False)
custom_embedding.weight.data[16] = pretrained_embedding.weight.data[token_id[0]]
print(f"Token ID for '<image_soft_token>': {token_id[0]}, new id: {16}")
token_id = pretrained_tokenizer.encode('<end_of_image>', add_special_tokens=False)
custom_embedding.weight.data[17] = pretrained_embedding.weight.data[token_id[0]]
print(f"Token ID for '<end_of_image>': {token_id[0]}, new id: {17}")
print("set up unused tokens")
custom_embedding.weight.data[18:24] = pretrained_embedding.weight.data[18:24]

Token ID for ' red': 2604
Token ID for ' blue': 3730
Token ID for ' green': 3826
Token ID for ' yellow': 7070
Token ID for ' orange': 11167
Token ID for ' purple': 16045
Token ID for ' white': 2173
Token ID for ' cyan': 49147
Token ID for ' gray': 12819
Token ID for ' brown': 8864
Token ID for '<bos>': 2, new id: 10
Token ID for ' input': 2744, new id: 11
Token ID for ' output': 3938, new id: 12
Token ID for '\n': 107, new id: 13
Token ID for '\n\n\n': 109, new id: 14
Token ID for '<start_of_image>': 255999, new id: 15
Token ID for '<image_soft_token>': 262144, new id: 16
Token ID for '<end_of_image>': 256000, new id: 17
set up unused tokens


In [5]:
torch.save(custom_embedding.state_dict(), '../../Model/gemma24.pth')

#### VLM

In [1]:
from functions import *
model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-pt-unsloth-bnb-4bit",
    max_seq_length = 8192, # Choose any for long context!
    load_in_4bit = True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-27 15:19:54 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.3.19: Fast Gemma3 patching. Transformers: 4.50.0.dev0. vLLM: 0.8.2.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.642 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [2]:
model.language_model.lm_head.load_state_dict(torch.load("/home/zhenlan/Desktop/Projects/ARC2/Model/lm_heads_weights_VLM.pth"))

<All keys matched successfully>

In [6]:
model.language_model.lm_head.weight.data[[2604, 3730, 3826, 7070, 11167, 16045, 2173, 49147, 12819, 8864, 2, 2744, 3938, 107, 109, 255999, 262144, 256000]].std(1).mean(), model.language_model.lm_head.weight.data[[2604, 3730, 3826, 7070, 11167, 16045, 2173, 49147, 12819, 8864, 2, 2744, 3938, 107, 109, 255999, 262144, 256000]].mean(1).mean()

(tensor(0.0173, device='cuda:0', dtype=torch.bfloat16),
 tensor(-2.6822e-05, device='cuda:0', dtype=torch.bfloat16))

In [7]:
custom_vocab_size = 48
pretrained_tokenizer = tokenizer.tokenizer
pretrained_model = model
pretrained_embedding = pretrained_model.get_input_embeddings()
embedding_dim = pretrained_embedding.embedding_dim
custom_embedding = nn.Embedding(custom_vocab_size, embedding_dim)
custom_embedding.weight.data.normal_(mean=0.0, std=0.0173)
colors = [" red", " blue", " green", " yellow", " orange", " purple", " white", " cyan", " gray", " brown"]
for i, color in enumerate(colors):
    # Encode the color directly
    token_id = pretrained_tokenizer.encode(color, add_special_tokens=False)
    assert len(token_id) == 1, f"Tokenization failed for {color}: {token_id}"
    print(f"Token ID for '{color}': {token_id[0]}")
    custom_embedding.weight.data[i] = pretrained_embedding.weight.data[token_id[0]]
token_id = pretrained_tokenizer.encode('<bos>', add_special_tokens=False)
custom_embedding.weight.data[10] = pretrained_embedding.weight.data[token_id[0]]
print(f"Token ID for '<bos>': {token_id[0]}, new id: {10}")
token_id = pretrained_tokenizer.encode(' input', add_special_tokens=False)
custom_embedding.weight.data[11] = pretrained_embedding.weight.data[token_id[0]]
print(f"Token ID for ' input': {token_id[0]}, new id: {11}")
token_id = pretrained_tokenizer.encode(' output', add_special_tokens=False)
custom_embedding.weight.data[12] = pretrained_embedding.weight.data[token_id[0]]
print(f"Token ID for ' output': {token_id[0]}, new id: {12}")
token_id = pretrained_tokenizer.encode('\n', add_special_tokens=False)
custom_embedding.weight.data[13] = pretrained_embedding.weight.data[token_id[0]]
print(f"Token ID for '\\n': {token_id[0]}, new id: {13}")
token_id = pretrained_tokenizer.encode('\n\n\n', add_special_tokens=False)
custom_embedding.weight.data[14] = pretrained_embedding.weight.data[token_id[0]]
print(f"Token ID for '\\n\\n\\n': {token_id[0]}, new id: {14}")
token_id = pretrained_tokenizer.encode('<start_of_image>', add_special_tokens=False)
custom_embedding.weight.data[15] = pretrained_embedding.weight.data[token_id[0]]
print(f"Token ID for '<start_of_image>': {token_id[0]}, new id: {15}")
token_id = pretrained_tokenizer.encode('<image_soft_token>', add_special_tokens=False)
custom_embedding.weight.data[16] = pretrained_embedding.weight.data[token_id[0]]
print(f"Token ID for '<image_soft_token>': {token_id[0]}, new id: {16}")
token_id = pretrained_tokenizer.encode('<end_of_image>', add_special_tokens=False)
custom_embedding.weight.data[17] = pretrained_embedding.weight.data[token_id[0]]
print(f"Token ID for '<end_of_image>': {token_id[0]}, new id: {17}")
print("All 48 tokens are active, no need to find tokens to have negative effect")

Token ID for ' red': 2604
Token ID for ' blue': 3730
Token ID for ' green': 3826
Token ID for ' yellow': 7070
Token ID for ' orange': 11167
Token ID for ' purple': 16045
Token ID for ' white': 2173
Token ID for ' cyan': 49147
Token ID for ' gray': 12819
Token ID for ' brown': 8864
Token ID for '<bos>': 2, new id: 10
Token ID for ' input': 2744, new id: 11
Token ID for ' output': 3938, new id: 12
Token ID for '\n': 107, new id: 13
Token ID for '\n\n\n': 109, new id: 14
Token ID for '<start_of_image>': 255999, new id: 15
Token ID for '<image_soft_token>': 262144, new id: 16
Token ID for '<end_of_image>': 256000, new id: 17
All 48 tokens are active, no need to find tokens to have negative effect


In [8]:
torch.save(custom_embedding.state_dict(), '../../Model/gemma48.pth')

#### Qwen 3

In [1]:
from functions import *
model, tokenizer = FastModel.from_pretrained(
                                        # model_name = "unsloth/gemma-3-4b-pt-unsloth-bnb-4bit",
                                        model_name = "unsloth/Qwen3-8B-unsloth-bnb-4bit",
                                        max_seq_length = 8192,
                                        load_in_4bit = True,
                                    )

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.7: Fast Qwen3 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.642 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
custom_vocab_size = 16

In [3]:
# pretrained_tokenizer = tokenizer.tokenizer
pretrained_tokenizer = tokenizer
pretrained_model = model

In [32]:
pretrained_embedding = pretrained_model.get_input_embeddings()
pretrained_bos_id = pretrained_tokenizer.encode("<|box_start|>", add_special_tokens=False)[0]
pretrained_eos_id = pretrained_tokenizer.encode("<|box_end|>", add_special_tokens=False)[0]
pretrained_line_break_id = pretrained_tokenizer.encode("\n", add_special_tokens=False)[0]

# Initialize custom embedding with the same embedding dimension as pre-trained
embedding_dim = pretrained_embedding.embedding_dim
custom_embedding = nn.Embedding(custom_vocab_size, embedding_dim)

# Set weights for special tokens
custom_embedding.weight.data[10] = pretrained_embedding.weight.data[pretrained_bos_id]  # BOS_X
custom_embedding.weight.data[11] = pretrained_embedding.weight.data[pretrained_eos_id]  # EOS_X
custom_embedding.weight.data[13] = pretrained_embedding.weight.data[pretrained_bos_id]  # BOS_Y
custom_embedding.weight.data[14] = pretrained_embedding.weight.data[pretrained_eos_id]  # EOS_Y
custom_embedding.weight.data[12] = pretrained_embedding.weight.data[pretrained_line_break_id]  # Line break

colors = [" red", " blue", " green", " yellow", " orange", " purple", " white", " cyan", " gray", " brown"]
for i, color in enumerate(colors):
    # Encode the color directly
    token_id = pretrained_tokenizer.encode(color, add_special_tokens=False)
    assert len(token_id) == 1, f"Tokenization failed for {color}: {token_id}"
    print(f"Token ID for '{color}': {token_id[0]}")
    custom_embedding.weight.data[i] = pretrained_embedding.weight.data[token_id[0]]

custom_embedding.weight.data[15] = pretrained_embedding.weight.data[pretrained_tokenizer.vocab_size - 1] # random non-special token

Token ID for ' red': 2518
Token ID for ' blue': 6303
Token ID for ' green': 6176
Token ID for ' yellow': 13753
Token ID for ' orange': 18575
Token ID for ' purple': 24932
Token ID for ' white': 4158
Token ID for ' cyan': 57888
Token ID for ' gray': 17545
Token ID for ' brown': 13876


In [33]:
custom_embedding.weight.std(1)

tensor([0.0279, 0.0270, 0.0270, 0.0262, 0.0259, 0.0251, 0.0273, 0.0241, 0.0256,
        0.0259, 0.0089, 0.0088, 0.0236, 0.0089, 0.0088, 0.0095],
       grad_fn=<StdBackward0>)

In [34]:
torch.save(custom_embedding.state_dict(), '../../Model/qwen16.pth')

#### Qwen 3 - 48

In [1]:
from functions import *
model, tokenizer = FastModel.from_pretrained(
                                        # model_name = "unsloth/gemma-3-4b-pt-unsloth-bnb-4bit",
                                        model_name = "unsloth/Qwen3-8B-unsloth-bnb-4bit",
                                        max_seq_length = 8192,
                                        load_in_4bit = True,
                                    )

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.7: Fast Qwen3 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.642 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
custom_vocab_size = 48

In [3]:
# pretrained_tokenizer = tokenizer.tokenizer
pretrained_tokenizer = tokenizer
pretrained_model = model

In [4]:
pretrained_embedding = pretrained_model.get_input_embeddings()
pretrained_bos_id = pretrained_tokenizer.encode("<|box_start|>", add_special_tokens=False)[0]
pretrained_eos_id = pretrained_tokenizer.encode("<|box_end|>", add_special_tokens=False)[0]
pretrained_line_break_id = pretrained_tokenizer.encode("\n", add_special_tokens=False)[0]

# Initialize custom embedding with the same embedding dimension as pre-trained
embedding_dim = pretrained_embedding.embedding_dim
custom_embedding = nn.Embedding(custom_vocab_size, embedding_dim)

# Set weights for special tokens
custom_embedding.weight.data[10] = pretrained_embedding.weight.data[pretrained_bos_id]  # BOS_X
custom_embedding.weight.data[11] = pretrained_embedding.weight.data[pretrained_eos_id]  # EOS_X
custom_embedding.weight.data[13] = pretrained_embedding.weight.data[pretrained_bos_id]  # BOS_Y
custom_embedding.weight.data[14] = pretrained_embedding.weight.data[pretrained_eos_id]  # EOS_Y
custom_embedding.weight.data[12] = pretrained_embedding.weight.data[pretrained_line_break_id]  # Line break

colors = [" red", " blue", " green", " yellow", " orange", " purple", " white", " cyan", " gray", " brown"]
for i, color in enumerate(colors):
    # Encode the color directly
    token_id = pretrained_tokenizer.encode(color, add_special_tokens=False)
    assert len(token_id) == 1, f"Tokenization failed for {color}: {token_id}"
    print(f"Token ID for '{color}': {token_id[0]}")
    custom_embedding.weight.data[i] = pretrained_embedding.weight.data[token_id[0]]

custom_embedding.weight.data[15] = pretrained_embedding.weight.data[pretrained_tokenizer.vocab_size - 1] # random non-special token

Token ID for ' red': 2518
Token ID for ' blue': 6303
Token ID for ' green': 6176
Token ID for ' yellow': 13753
Token ID for ' orange': 18575
Token ID for ' purple': 24932
Token ID for ' white': 4158
Token ID for ' cyan': 57888
Token ID for ' gray': 17545
Token ID for ' brown': 13876


In [7]:
for i in range(1, 31):
    # Encode the integer directly
    txt = str(i % 10) # Assuming single token output for each integer. only true for 1~9, hence %.
    token_id = pretrained_tokenizer.encode(txt, add_special_tokens=False)
    assert len(token_id) == 1, f"Tokenization failed for {txt}: {token_id}"
    custom_embedding.weight.data[17 + i] = pretrained_embedding.weight.data[token_id[0]]

In [ ]:
custom_embedding.weight.data[16] = custom_embedding.weight.data[18:].mean(0)
custom_embedding.weight.data[17] = custom_embedding.weight.data[18:].median(0).values

In [10]:
# custom_embedding.weight.data[16:18].div_(1/0.0258);
# custom_embedding.weight.data[18:].median(0).values;

tensor([[ 0.0426,  0.0204, -0.0201,  ...,  0.0363, -0.0052, -0.0250],
        [-0.0018,  0.0293, -0.0065,  ..., -0.0048,  0.0051,  0.0103]])

In [16]:
custom_embedding.weight.std(1)

tensor([0.0279, 0.0270, 0.0270, 0.0262, 0.0259, 0.0251, 0.0273, 0.0241, 0.0256,
        0.0259, 0.0089, 0.0088, 0.0236, 0.0089, 0.0088, 0.0095, 0.0176, 0.0172,
        0.0245, 0.0241, 0.0247, 0.0248, 0.0246, 0.0245, 0.0258, 0.0256, 0.0252,
        0.0230, 0.0245, 0.0241, 0.0247, 0.0248, 0.0246, 0.0245, 0.0258, 0.0256,
        0.0252, 0.0230, 0.0245, 0.0241, 0.0247, 0.0248, 0.0246, 0.0245, 0.0258,
        0.0256, 0.0252, 0.0230], grad_fn=<StdBackward0>)

In [17]:
torch.save(custom_embedding.state_dict(), '../../Model/qwen48.pth')